Few Shot Prompt Templates
- 目的のリクエストの前にインプットとアウトプットの例を幾つかLLMに提示する
- ベストプラクティスについてコンセンサスはないがLangChainではhuman messageとAI messageのインプットをビルドすることをリコメンド

https://python.langchain.com/v0.1/docs/modules/model_io/prompts/few_shot_examples_chat/

langchainのマニュアルではチャットモデルでの説明とcompletionモデルでの説明がある。

以下langchainのチャットモデルでの例(コースだとmathのところの内容に近い)

In [1]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

In [2]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [3]:
# This is a prompt template used to format each individual example.
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [4]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [5]:
# from langchain_community.chat_models import ChatAnthropic
# Anthropicの無料モデルはでAPIは使えなさそう
from langchain_google_genai import ChatGoogleGenerativeAI

chain = final_prompt | ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0)


In [6]:
input_str = '"input": "What\'s the square of a triangle?"'
input_str = '"input": "three plus two?"'
print(input_str)
result = chain.invoke({input_str})
print(result.content)

"input": "three plus two?"
5 



langchainサンプル続き

python 3.12でchroma導入が失敗(正確にはhnswlibの導入でエラー)したためpython 3.10で環境を再構成。

langchainのサンプルで行っていること
- examples準備。計算、QA、命令等混ざっている
- embeggings設定。サンプルではOpenAIEmbeddings()がセットされる
- Chroma.from_texts()でstore to vectorstore
  
- SemanticSimilarityExampleSelectorでvectorstoreとk=2(類似は2つ返して)をセットしexample_selector作成
- その後インプットにhorseをセットして実行。cowのエントリーが一番に返される

- 更にFewShotChatMessagePromptTemplate()でfew_shot_promptを定義
- その後幾つかの実行例



In [7]:
from langchain_chroma import Chroma

from langchain_core.example_selectors import SemanticSimilarityExampleSelector

# from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [8]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
# embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [16]:
print(to_vectorize)

['2+2 4', '2+3 5', '2+4 6', 'What did the cow say to the moon? nothing at all', 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']


エラーは無いが、horseで2+3が返される。

In [9]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})
# example_selector.select_examples({"input": "what did you say"})

[{'input': '2+3', 'output': '5'}, {'input': '2+2', 'output': '4'}]

In [10]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)

In [11]:
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [12]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

In [13]:
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [14]:
# from langchain_community.chat_models import ChatAnthropic

chain = final_prompt # | ChatAnthropic(temperature=0.0)

result = chain.invoke({"input": "What's 3+3?"})

In [15]:
print(result.to_string())

System: You are a wondrous wizard of math.
Human: 2+3
AI: 5
Human: 2+2
AI: 4
Human: What's 3+3?


以下はコースの例

In [ ]:
# from langchain_openai.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
template = "You are a helpful assistant that translates complex legal terms into plain and understandable terms."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

legal_text = "The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."
example_input_one = HumanMessagePromptTemplate.from_template(legal_text)

plain_text = "The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law."
example_output_one = AIMessagePromptTemplate.from_template(plain_text)

human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_one, example_output_one, human_message_prompt]
)

In [ ]:
some_example_text = "The grantor, being the fee simple owner of the real property herein described, conveys and warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, and any applicable covenants, conditions, and restrictions affecting the property, in consideration of the sum of [purchase price] paid by the grantee."
request = chat_prompt.format_prompt(legal_text=some_example_text).to_messages()


In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash",convert_system_message_to_human=True)
    

In [ ]:
result = chat.invoke(request)

In [ ]:
print(result.content)